In [ ]:
import win32com.client
import pdb
from operator import itemgetter
from tqdm import tqdm
import csv

o = win32com.client.gencache.EnsureDispatch("Outlook.Application")
ns = o.GetNamespace("MAPI")

adrLi = ns.AddressLists.Item("Global Address List")
contacts = adrLi.AddressEntries
numEntries = adrLi.AddressEntries.Count

export_contacts = list()

COLUMNS = ('full_name', 'first_name', 'last_name', 'email')
FIELDS = [itemgetter(field) for field in COLUMNS]

def to_contacts(contact: any) -> dict:
    if "," not in contact.Name:
        return None
    if contact.Type == 'EX':
        eu = contact.GetExchangeUser()
        if not eu:
            return None
        email_address = eu.PrimarySmtpAddress
    elif contact.Type == 'SMTP':
        email_address = contact.Address
    else:
        email_address = None
    if not email_address:
        return None
    try:
        return dict(
            full_name=contact.Name,
            first_name=eu.FirstName if eu else None,
            last_name=eu.LastName if eu else None,
            email=email_address
        )
    except Exception as e:
        return None
    
def persons_only(contact) -> bool:
    return bool(contact) and isinstance(contact, dict)

persons = filter(persons_only, map(to_contacts, contacts))
csv_file = "contacts.csv"    

with open(csv_file, "w", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=COLUMNS, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writeheader()
    for contact in tqdm(persons):
        writer.writerow(contact)


export_contacts

70910it [1:07:10, 30.33it/s]